# Titanic - Machine Learning from Disaster
### 타이타닉호 생존자 분석하기


In [1]:
import pandas as pd
import numpy as np

#아래의 코드를 실행하지않으면 로지스틱회귀의 에러구문 반환을 구경하게된다.
import warnings
warnings.filterwarnings('ignore')

In [2]:
#dataset과 script를 같은 위치에 놓고 시작함.
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

위의 정보를 통해 train data는 생존 여부를 포함한다는 것을 알 수 있다.  
각각의 특성들이 가지는 의미를 알아보자.  
- Pclass: 좌석등급을 의미한다. 
- SibSp: 탑승한 (형제자매+배우자)의 수를 의미한다.
- Parch: 탑승한 (부모+자녀)의 수를 의미한다.
- Cabin: 객실번호를 의미한다.
- Fare: 탑승요금을 의미한다.
- Embarkd: 탑승항구를 의미한다.
- Ticket: 티켓번호를 의미한다.

Passengerld,Name,Ticket은 무의미한 데이터라고 생각했기 때문에 drop으로 제거했다.

In [4]:
train = train.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
test = test.drop(['Name','Ticket'], axis=1)

train.head

<bound method NDFrame.head of      Survived  Pclass     Sex   Age  SibSp  Parch     Fare Cabin Embarked
0           0       3    male  22.0      1      0   7.2500   NaN        S
1           1       1  female  38.0      1      0  71.2833   C85        C
2           1       3  female  26.0      0      0   7.9250   NaN        S
3           1       1  female  35.0      1      0  53.1000  C123        S
4           0       3    male  35.0      0      0   8.0500   NaN        S
..        ...     ...     ...   ...    ...    ...      ...   ...      ...
886         0       2    male  27.0      0      0  13.0000   NaN        S
887         1       1  female  19.0      0      0  30.0000   B42        S
888         0       3  female   NaN      1      2  23.4500   NaN        S
889         1       1    male  26.0      0      0  30.0000  C148        C
890         0       3    male  32.0      0      0   7.7500   NaN        Q

[891 rows x 9 columns]>

※데이터 간의 상관관계를 파악 후 진행하는게 맞지만 데이터를 다루는데 아직 미숙한 부분도 있고 정보도 확실하지않아 본 글을 읽을 시 유의가 필요하다..  
먼저 범주형 데이터를 처리하기위해 one-hot-encoding을 진행한다.

Pclass는 연속적 데이터가 아니라 범주형 데이터로 보아야한다.

In [5]:
pclass_train_dummies = pd.get_dummies(train['Pclass'])
pclass_test_dummies = pd.get_dummies(test['Pclass'])

train.drop(['Pclass'], axis=1, inplace=True)
test.drop(['Pclass'], axis=1, inplace=True)

train = train.join(pclass_train_dummies)
test = test.join(pclass_test_dummies)

In [6]:
sex_train_dummies = pd.get_dummies(train['Sex'])
sex_test_dummies = pd.get_dummies(test['Sex'])

sex_train_dummies.columns = ['Female', 'Male']
sex_test_dummies.columns = ['Female', 'Male']

train.drop(['Sex'], axis=1, inplace=True)
test.drop(['Sex'], axis=1, inplace=True)

train = train.join(sex_train_dummies)
test = test.join(sex_test_dummies)

In [7]:
print(train.isnull().sum())
print(test.isnull().sum())

test['Fare'].fillna(0,inplace=True)

Survived      0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
1             0
2             0
3             0
Female        0
Male          0
dtype: int64
PassengerId      0
Age             86
SibSp            0
Parch            0
Fare             1
Cabin          327
Embarked         0
1                0
2                0
3                0
Female           0
Male             0
dtype: int64


아래의 정보를 보다싶이 Age에는 결측값이 존재하는데 이를 평균값으로 메꾸려고한다.  

In [8]:
import matplotlib.pyplot as plt

is_age=train.loc[(train['Age'].isnull()),['Survived','Age']]
is_age['Survived'].value_counts()

0    125
1     52
Name: Survived, dtype: int64

In [9]:
train["Age"].fillna(train["Age"].mean() , inplace=True)
test["Age"].fillna(test["Age"].mean() , inplace=True)

객실의 유무가 유의미한 영향이 있다고 판단되지만 대부분 NaN값이라 전처리하기 애매한 부분이 있다고 생각했고 탑승항구 또한 결과에 유의미한 영향을 미칠것이라고 생각하지않아서 둘 다 drop을 실행했다.

In [10]:
train=train.drop(['Cabin','Embarked'],axis=1)
test=test.drop(['Cabin','Embarked'],axis=1)

In [11]:
train.head()
test.head()

,PassengerId,Age,SibSp,Parch,Fare,1,2,3,Female,Male
0,892,34.5,0,0,7.8292,0,0,1,0,1
1,893,47.0,1,0,7.0000,0,0,1,1,0
2,894,62.0,0,0,9.6875,0,1,0,0,1
3,895,27.0,0,0,8.6625,0,0,1,0,1
4,896,22.0,1,1,12.2875,0,0,1,1,0


이제 학습을 실시하기위해 데이터를 나누어야한다.

In [12]:
x_train=train.drop("Survived",axis=1)
y_train=train['Survived']
x_test=test.drop("PassengerId",axis=1)

머신러닝 모듈을 불러오고 어떤 모델이 가장 점수가 높은지 확인한다. 

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

#로지스틱회귀
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)
print(f"LogisticRegression: {log_reg.score(x_train, y_train)}")

#서포트벡터머신
svc = SVC()
svc.fit(x_train, y_train)
print(f"SupportVectorMachine: {svc.score(x_train, y_train)}")

#랜덤포레스트
random_forest=RandomForestClassifier(n_estimators=30)
random_forest.fit(x_train, y_train)
print(f"RandomForest: {random_forest.score(x_train, y_train)}")

#K최근접이웃
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train, y_train)
print(f"KNN: {knn.score(x_train, y_train)}")


LogisticRegression: 0.8013468013468014
SupportVectorMachine: 0.6868686868686869
RandomForest: 0.9820426487093153
KNN: 0.8462401795735129


랜덤포레스트가 가장 좋은 점수를 내놓는 것을 확인할 수 있다.

최종적으로 랜덤포레스트를 사용해서 제출파일을 만든다.

In [14]:
random_forest=RandomForestClassifier(n_estimators=30)
random_forest.fit(x_train, y_train)
y_pred=random_forest.predict(x_test)

result=pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": y_pred
})
result.to_csv('result.csv',index=False)


score는 0.75598이다.